In [77]:
from TextNormalizer import TextNormalizer
from flashtext import KeywordProcessor
import pickle

In [152]:
def save_bin(fname,var):
    with open(fname,'wb') as fp:
        pickle.dump(var,fp)

In [74]:
def load_bin(fname):
    with open(fname,'rb') as fp:
        return pickle.load(fp)

In [86]:
def load_txt(fname,vec_delim = '/',phrase_delim='\n'):
    with open(fname) as f:
        return [list(filter(lambda x: x!='',elem.split(phrase_delim))) for elem in f.read().split(vec_delim)]

In [103]:
def normalize_vecs(vecs):
    return [[' '.join(TextNormalizer.normalize(phrase)) for phrase in vec] for vec in vecs]

In [104]:
def normalize_docs(docs):
    return [' '.join(TextNormalizer.normalize(doc)) for doc in docs]

In [154]:
nvecs = load_bin('nvecs')
ndocs = load_bin('ndocs')

In [143]:
def relevance(vec,doc):
    keyword_processor = KeywordProcessor()
    for phrase in vec:
        keyword_processor.add_keyword(phrase)
    found_phrases = keyword_processor.extract_keywords(doc)
    relevance = len(found_phrases)/len(vec)
    missing_phrases = set(vec).difference(set(found_phrases))
    return relevance, missing_phrases

In [155]:
relevance(nvecs[1],ndocs[1])

(0.2,
 {'обслуживание бытовой кондиционер',
  'обслуживание кондиционер daikin',
  'обслуживание кондиционер lg',
  'обслуживание кондиционер mitsubishi',
  'обслуживание кондиционер panasonic',
  'обслуживание кондиционер работа',
  'обслуживание кондиционер сплит система',
  'обслуживание кондиционер цена',
  'обслуживание сплит система',
  'прайс обслуживание кондиционер',
  'разовый обслуживание кондиционер',
  'расценка обслуживание кондиционер',
  'ремонт бытовой кондиционер',
  'ремонт кондиционер цена',
  'ремонт обслуживание кондиционер',
  'ремонт установка кондиционер',
  'сервисный обслуживание кондиционер цена',
  'сколько стоить обслуживание кондиционер',
  'срочный ремонт кондиционер',
  'стоимость обслуживание кондиционер',
  'технический обслуживание кондиционер стоимость',
  'технический обслуживание кондиционер цена',
  'техобслуживание кондиционер',
  'чистка кондиционер'})

### Tf-idf

In [17]:
nvecs = [TextNormalizer.normalize(text) for text in vecs]

In [18]:
ndocs = [TextNormalizer.normalize(doc) for doc in docs]

In [19]:
dct = Dictionary(nvecs)

In [20]:
vec_corpus = [dct.doc2bow(vec) for vec in nvecs]

In [21]:
doc_corpus = [dct.doc2bow(doc) for doc in ndocs]

In [23]:
model = TfidfModel(vec_corpus,dictionary=dct)

In [24]:
tf_vecs = [model[vec] for vec in vec_corpus]

In [25]:
tf_docs = [model[doc] for doc in doc_corpus]

In [26]:
index = similarities.SparseMatrixSimilarity(tf_vecs, num_features=dct.num_pos)

In [36]:
index[tf_docs[3]]

array([0.07533741, 0.06170711, 0.10711007, 0.78122646], dtype=float32)